Plan:

* Graph definition
* Graph vizualization
* Graph representation in memory
* Graph problems (loops, strongly connected components, flows)
* Graph algorithms (traversal, search, path)

## Naive graph representation:

In [3]:
from typing import NamedTuple, List, Tuple

In [4]:
class Graph(NamedTuple):
    nodes: List[str]
    edges: List[Tuple[str, str]]


In [5]:
konigsberg_nodes = ["A", "B", "C", "D"]

In [6]:
konigsberg_edges = [
    ("A", "B"),
    ("A", "B"),
    ("A", "C"),
    ("A", "C"),
    ("A", "D"),
    ("B", "D"),
    ("C", "D"),
]

In [10]:
konigsberg_bridges = Graph(konigsberg_nodes, konigsberg_edges)

## Linked objects representation:

## Adjacency list (список суміжності)

Це спосіб репрезентації зв'язків у графі через створення записів у словнику виду:

Node: [Node1, Node2, Node3... NodeM]. В списку знаходяться ноди, від котрих до Node існує ребро (edge)

В Python правильний це adjacency dict. Звідси висновок: об'єкт, котрий в нас виступає в ролі ноди, має бути hashable.

Також у системах, де можливо робити декілька записів для одного і того самого ключа, ви можете розгорнути adjacency list у набір записів типу NodeM: Node1, NodeM: Node2, etc (наприклад у RDBMS)

In [11]:
def get_adjacency_list(graph: Graph):
    """
    Return the adjacency list representation
    of the graph.
    """
    adj = {node: list() for node in graph.nodes}
    for edge in graph.edges:
        node_1, node_2 = edge[0], edge[1]
        adj[node_1].append(node_2)
        adj[node_2].append(node_1)
    return adj

In [12]:
get_adjacency_list(konigsberg_bridges)

{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [17]:
{edge: index for index, edge in enumerate(konigsberg_bridges.nodes)}

{'A': 0, 'B': 1, 'C': 2, 'D': 3}

## Adjacency matrix (матриця суміжності)

In [13]:
import numpy as np

def get_adjacency_matrix(graph):
    """
    Returns the adjacency matrix of the graph.
    """
    num_nodes = len(graph.nodes)
    edges_to_index = {edge: index for index, edge in enumerate(konigsberg_bridges.nodes)}
    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=np.int8)
    
    for edge in graph.edges:
        node_1, node_2 = edge[0], edge[1]
        
    

## Incidence matrix